# Day 7

## Part 1

Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

- Five of a kind, where all five cards have the same label: AAAAA
- Four of a kind, where four cards have the same label and one card has a different label: AA8AA
- Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
- Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
- Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
- One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
- High card, where all cards' labels are distinct: 23456
- Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:

```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

- 32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
- KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
- T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.
Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [ ]:
from collections import Counter


CARD_RANKS = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]


class Card:
    def __init__(self, value: str):
        self.value = value

    def __eq__(self, other: "Card") -> bool:
        return self.value == other.value

    def __gt__(self, other: "Card") -> bool:
        return self.score > other.score

    def __lt__(self, other: "Card") -> bool:
        return self.score < self.score

    def __repr__(self) -> str:
        return f"<Card {self.value!r}>"

    @property
    def score(self) -> int:
        return CARD_RANKS.index(self.value)


HAND_RANKS = ["HC", "1P", "2P", "3K", "FH", "4K", "5K"]


class CardHand:
    def __init__(self, hand_data: str):
        self.raw_cards, bid = hand_data.split()
        self.bid = int(bid)
        self.cards: list[Card] = [Card(value) for value in self.raw_cards]

        self.card_counts: dict[str, int] = Counter(card.value for card in self.cards)

        self.one_pair = [count for count in self.card_counts.values() if count == 2]
        self.two_pair = len(self.one_pair) == 2
        self.three_of_a_kind = [count for count in self.card_counts.values() if count == 3]
        self.full_house = self.one_pair and self.three_of_a_kind
        self.four_of_a_kind = [count for count in self.card_counts.values() if count == 4]
        self.five_of_a_kind = [count for count in self.card_counts.values() if count == 5]

    @property
    def rank(self) -> str:
        if self.five_of_a_kind:
            return "5K"
        elif self.four_of_a_kind:
            return "4K"
        elif self.full_house:
            return "FH"
        elif self.three_of_a_kind:
            return "3K"
        elif self.two_pair:
            return "2P"
        elif self.one_pair:
            return "1P"
        else:
            return "HC"

    @property
    def rank_score(self) -> int:
        hand_score = HAND_RANKS.index(self.rank) * 1_000_000_000
        hand_score += self.cards[0].score * 10_000_000
        hand_score += self.cards[1].score * 100_000
        hand_score += self.cards[2].score * 1_000
        hand_score += self.cards[3].score * 10
        hand_score += self.cards[4].score
        return hand_score

    def __repr__(self) -> str:
        return f"<CardHand {self.raw_cards}: {self.rank} (score={self.rank_score})>"

    def __eq__(self, other: "CardHand") -> bool:
        return self.cards == other.cards

    def __gt__(self, other: "CardHand") -> bool:
        for card, other_card in zip(self.cards, other.cards):
            if card == other_card:
                continue
            return card > other_card

    def __lt__(self, other: "CardHand") -> bool:
        for card, other_card in zip(self.cards, other.cards):
            if card == other_card:
                continue
            return card < other_card


def calculate_winnings(hands) -> int:
    winnings = 0
    for rank, hand in enumerate(hands):
        rank_multiplier = rank + 1
        hand_winning = hand.bid * rank_multiplier
        print(f"{hand} ({hand.bid=} x {rank_multiplier=} = {hand_winning=})")
        winnings += hand.bid * rank_multiplier
    return winnings


test_inputs = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".splitlines()

test_hands = sorted(
    [CardHand(test_input) for test_input in test_inputs], key=lambda x: x.rank_score
)
test_winnings = calculate_winnings(test_hands)
assert test_winnings == 6440, test_winnings

In [2]:
test_winnings

6440

In [ ]:
inputs = open("../inputs/7.txt").read().splitlines()

hands = sorted([CardHand(input_) for input_ in inputs], key=lambda x: x.rank_score)

winnings = calculate_winnings(hands)

In [4]:
winnings

249483956

## Part 2

To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
- 32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
- KK677 is now the only two pair, making it the second-weakest hand.
- T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.

With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [ ]:
CARD_RANKS_WITH_JOKERS = ["J", "2", "3", "4", "5", "6", "7", "8", "9", "T", "Q", "K", "A"]


class Card:
    def __init__(self, value: str):
        self.value = value

    def __eq__(self, other: "Card") -> bool:
        return self.value == other.value

    def __gt__(self, other: "Card") -> bool:
        return self.score > other.score

    def __lt__(self, other: "Card") -> bool:
        return self.score < self.score

    def __repr__(self) -> str:
        return f"<Card {self.value!r}>"

    @property
    def score(self) -> int:
        return CARD_RANKS_WITH_JOKERS.index(self.value)


class CardHandWithJokers:
    def __init__(self, hand_data: str):
        self.raw_cards, bid = hand_data.split()
        self.bid = int(bid)
        self.cards: list[Card] = [Card(value) for value in self.raw_cards]

        self.card_counts: dict[str, int] = Counter(card.value for card in self.cards)
        self.adjust_counts_for_jokers()

        self.one_pair = [count for count in self.card_counts.values() if count == 2]
        self.two_pair = len(self.one_pair) == 2
        self.three_of_a_kind = [count for count in self.card_counts.values() if count == 3]
        self.full_house = self.one_pair and self.three_of_a_kind
        self.four_of_a_kind = [count for count in self.card_counts.values() if count == 4]
        self.five_of_a_kind = [count for count in self.card_counts.values() if count == 5]

    def adjust_counts_for_jokers(self):
        self.joker_count = self.card_counts.pop("J", 0)
        if self.joker_count == 0:
            # no jokers, no adjustments
            return

        if self.joker_count == 5:
            # all jokers, max score
            self.card_counts = {"A": 5}
            return

        top_count = max(self.card_counts.values())
        top_count_cards = [card for card, count in self.card_counts.items() if count == top_count]
        if len(top_count_cards) == 1:
            # one non-Joker card is the top count, add jokers to it
            self.card_counts[top_count_cards[0]] += self.joker_count
            return

        # multiple top count cards, add jokers to the highest value card (2P)
        unique_cards = set(top_count_cards)
        higher_card = max(unique_cards, key=lambda x: CARD_RANKS_WITH_JOKERS.index(x))
        self.card_counts[higher_card] += self.joker_count

    @property
    def rank(self) -> str:
        if self.five_of_a_kind:
            return "5K"
        elif self.four_of_a_kind:
            return "4K"
        elif self.full_house:
            return "FH"
        elif self.three_of_a_kind:
            return "3K"
        elif self.two_pair:
            return "2P"
        elif self.one_pair:
            return "1P"
        else:
            return "HC"

    @property
    def rank_score(self) -> int:
        hand_score = HAND_RANKS.index(self.rank) * 1_000_000_000
        hand_score += self.cards[0].score * 10_000_000
        hand_score += self.cards[1].score * 100_000
        hand_score += self.cards[2].score * 1_000
        hand_score += self.cards[3].score * 10
        hand_score += self.cards[4].score
        return hand_score

    def __repr__(self) -> str:
        return f"<CardHand {self.raw_cards}: {self.rank} (score={self.rank_score})>"

    def __eq__(self, other: "CardHand") -> bool:
        return self.cards == other.cards

    def __gt__(self, other: "CardHand") -> bool:
        for card, other_card in zip(self.cards, other.cards):
            if card == other_card:
                continue
            return card > other_card

    def __lt__(self, other: "CardHand") -> bool:
        for card, other_card in zip(self.cards, other.cards):
            if card == other_card:
                continue
            return card < other_card


test_hands2 = sorted(
    [CardHandWithJokers(test_input) for test_input in test_inputs], key=lambda x: x.rank_score
)

test_winnings2 = calculate_winnings(test_hands2)
assert test_winnings2 == 5905, test_winnings2

In [6]:
test_winnings2

5905

In [ ]:
hands2 = sorted([CardHandWithJokers(input_) for input_ in inputs], key=lambda x: x.rank_score)

winnings2 = calculate_winnings(hands2)

In [8]:
winnings2

252137472